In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from tqdm import tqdm
from ln_utils import handle_img, load_data
from transaction_simulator import get_experiment_files

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set(font_scale = 2)
sns.set_style("whitegrid")

# 1. Load path info files

In [ ]:
snapshots = range(7)

In [ ]:
results_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/path_length_info/"

### Experiments with capacity filter

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_7days/"
amounts = [10000,25000,50000,100000,200000,500000]
tx_nums = [1000,2000,5000,10000,20000]
ts_map = {
    1000:"2019-09-16_20:45:51",
    2000:"2019-09-16_19:13:21",
    5000:"2019-09-16_14:12:54",
    10000:"2019-09-16_17:56:28",
    20000:"2019-09-16_15:39:13"
}

### Experiments without capacity filter

In [ ]:
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations"
amounts = [10000,25000,50000,100000,200000,500000]
tx_nums = [1000,2000,5000,10000,20000,50000]
ts_map = {
    1000:"2019-09-23_22:53:58",
    2000:"2019-09-23_15:29:59",
    5000:"2019-09-23_06:47:40",
    10000:"2019-09-22_22:17:29",
    20000:"2019-09-21_23:53:33",
    50000:"2019-09-23_21:37:58"
}

### Load simulation outputs

In [ ]:
records = []
for n in tqdm(tx_nums):
    for a in amounts:
        experiment_id = "%s_%isat_k%i" % (ts_map[n], a, n)
        experiment_folders = get_experiment_files(experiment_id, snapshots, simulation_dir)
        length_distrib = load_data(experiment_folders, snapshots, "lengths_distrib")
        all_length_distrib = pd.concat(length_distrib)
        all_success = all_length_distrib[all_length_distrib["length"]>0]
        avg_path_length = all_success["length"].mean()
        total_counts = all_length_distrib.groupby("length")["count"].sum()
        S = total_counts.sum()
        total_ratios = total_counts / S
        ratio_failed = total_ratios[-1]
        records.append([n,a,ratio_failed,avg_path_length])
        #print(n,a)

# Transaction failure and average path length

## Pivot tables

In [ ]:
ratio_col = "ratio of failed transactions"
avg_path_col = "average path length"
tx_num_col = "number of transactions"

In [ ]:
path_info_df = pd.DataFrame(records, columns=[tx_num_col,"amount",ratio_col,avg_path_col])

In [ ]:
path_info_df.head()

In [ ]:
pivot_ratio_failed = path_info_df.pivot(index="amount",columns=tx_num_col,values=ratio_col).sort_values("amount",ascending=False)

In [ ]:
pivot_ratio_avg_length = path_info_df.pivot(index="amount",columns=tx_num_col,values=avg_path_col).sort_values("amount",ascending=False)

## Heatmaps

In [ ]:
cmap = "RdBu_r"
fig_s=(6,6)

In [ ]:
pivot_ratio_failed.shape

In [ ]:
pivot_ratio_failed

In [ ]:
fig, ax = plt.subplots(figsize=fig_s)
sns.heatmap(data=pivot_ratio_failed, ax=ax, cmap=cmap, square=True)
ax.set_ylim(6.0, 0)
plt.savefig("%s/ratio_of_failed_paths.pdf" % results_dir, format="pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=fig_s)
sns.heatmap(data=pivot_ratio_avg_length, ax=ax, cmap=cmap,  square=True)
ax.set_ylim(6.0, 0)
plt.savefig("%s/avg_path_length.pdf" % results_dir, format="pdf", bbox_inches='tight')

# Transaction security analysis

In [ ]:
n = 5000
a = 50000
experiment_id = "%s_%isat_k%i" % (ts_map[n], a, n)
experiment_folders = get_experiment_files(experiment_id, snapshots, simulation_dir)
length_distrib = load_data(experiment_folders, snapshots, "lengths_distrib")
all_length_distrib = pd.concat(length_distrib)

In [ ]:
all_length_distrib.head()

In [ ]:
success_transactions = all_length_distrib.copy()[all_length_distrib["length"]>0]

In [ ]:
success_transactions["length"] = success_transactions["length"].apply(lambda x: x if x<6 else "longer")

success_transactions.loc[:,"ratio"] = success_transactions["count"] / 5000

In [ ]:
plt.figure(figsize=(6,6))
labs = [1,2,3,4,5,'longer']
values = success_transactions.groupby("length")["count"].sum()
plt.pie(values, explode=(0.2,0,0,0,0.2,0.2), autopct='%1.0f%%', pctdistance=1.17)
lgd = plt.legend(labs, loc=2,bbox_to_anchor=(1.08, 1.02))
plt.savefig("%s/path_length_distribution.pdf" % results_dir, format="pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')

sns.catplot(data=success_transactions, x="length", y="ratio", kind="bar", height=8)